In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import re
import pandas as pd
import numpy as np
#from scipy.special import gamma as GammaFunction
import pymc as pm
import arviz as az
import bambi as bmb
import pytensor.tensor as at
import matplotlib.pyplot as plt
import seaborn as sns
import attila_utils
import mwm

In [2]:
%connect_info

{
  "shell_port": 55498,
  "iopub_port": 55499,
  "stdin_port": 55500,
  "control_port": 55502,
  "hb_port": 55501,
  "ip": "127.0.0.1",
  "key": "36a5474d-5678cbc3965da755e42aed42",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": "python3"
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-620e67fb-0808-4b61-9ec1-14467882a054.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
